In [2]:
# Main file that need help with.
# Input -> Presidio -> Detected Entities -> GPT-4o -> Metric calculations
# Only do WITHIN context filteration (need to extract the sentence where the detected entity belongs to and feed to GPT).
# No need to do out-of-context filteration
import ast
import json
from openai import OpenAI
# Load the API key from input/secret.json
# secret_json = read_json_file('input/secret.json')
with open('input/secret.json', 'r', encoding='UTF-8') as file:
    secret_json = json.load(file)
api_key = secret_json['OPENAI_API_KEY']
client = OpenAI(api_key=api_key)

In [6]:
# TODO: Incomplete function mainly written by ChatGPT, haven't debugged yet.
# Be aware of the generated cost.

def check_pii_entities_gpt4o(detected_entities):
    # Separate entities by category
    person_entities = [entity for entity in detected_entities if entity[2] == 'PERSON']
    email_entities = [entity for entity in detected_entities if entity[2] == 'EMAIL_ADDRESS']
    url_entities = [entity for entity in detected_entities if entity[2] == 'URL']
    phone_entities = [entity for entity in detected_entities if entity[2] == 'PHONE_NUMBER']

    # Function to call GPT-4o and check for PII
    def check_entities_with_gpt(entities, category, prompt_template):
        # Prepare input text for GPT by batching entities
        prompt = prompt_template + "\n\n"
        for entity in entities:
            idx, entity_text, _, positions = entity
            prompt += f"{entity_text}\n"

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        # TODO: Possibly buggy lines below.
        # gpt_results = response.choices[0].message.content
        gpt_results = response.choices[0].message.content.strip().split('\n')

        # Process the response and keep entities marked as True (PII detected)
        filtered_entities = [
            entity for entity, result in zip(entities, gpt_results) if result.strip().upper() == 'T'
        ]
        return filtered_entities

    # Define prompt templates for each category
    # TODO: Feel free to change the prompt if necessary.
    person_prompt = "For each of the following texts, please decide if it is a student's name (return 'T' for True or 'F' for False):"
    email_prompt = "For each of the following texts, please decide if it is a personal email address (return 'T' for True or 'F' for False):"
    url_prompt = "For each of the following texts, please decide if it is a personal URL (return 'T' for True or 'F' for False):"
    phone_prompt = "For each of the following texts, please decide if it is a personal phone number (return 'T' for True or 'F' for False):"

    # Process each category with GPT-4o
    valid_person_entities = check_entities_with_gpt(person_entities, 'PERSON', person_prompt)
    valid_email_entities = check_entities_with_gpt(email_entities, 'EMAIL_ADDRESS', email_prompt)
    valid_url_entities = check_entities_with_gpt(url_entities, 'URL', url_prompt)
    valid_phone_entities = check_entities_with_gpt(phone_entities, 'PHONE_NUMBER', phone_prompt)

    # Combine all valid entities into a single list
    valid_entities = valid_person_entities + valid_email_entities + valid_url_entities + valid_phone_entities

    # TODO: Might need to sort based on file_idx (first item in tuple) before return.
    return valid_entities


detected_entities = []
# with open('output/pii_detected_trf_filtered.txt', 'r') as file:
#     for line in file:
#         entity = ast.literal_eval(line.strip())
#         detected_entities.append(entity)

with open('output/pii_detected_trf_filtered.txt', 'r') as file:
        for line in file:
            entity = ast.literal_eval(line.strip())
            idx, _, _, _ = entity
            if idx < 100: # Start from a few examples to try
                detected_entities.append(entity)

# Call the function to filter valid entities
filtered_entities = check_pii_entities_gpt4o(detected_entities)
print(f"Filtered entities: {filtered_entities}")

# TODO: Possibly save the filtered out entities in 'output/pii_detected_gpt.txt' or any suitable file name.
# with open('output/pii_detected_gpt.txt', 'w') as f:
#      ...


Filtered entities: []


In [ ]:
# When you finish, please go to evaluator.ipynb and run  calculate_metrics('output/pii_detected_gpt.txt')
# and go to evaluator_categories.ipynb and run  evaluate_detected_entities('output/pii_detected_gpt.txt')
# The two lines of code are marked with TODO.